<a href="https://colab.research.google.com/github/sakamototaisei/python_colab/blob/main/face_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **顔認識の仕組み**

## 画像認識とは

*   画像認識とは、その画像が特定のカテゴリーのどれを表しているか識別すること
*   SVM：決定教会を用いて分類する
*   デープラーニングによる分類機で分類


## 顔認識の方法



*   顔認識は、デジタル写真から検出された顔画像を、登録された人物と照合させて識別すること
*   顔認証は、画像の人物が登録された人物と同一人物であるか判定して、公の期間が証明すること



**顔認識の処理フロー**



## 顔の見分け方：ランドマーク



*   顔の特徴を学習して距離を測る(目、眉、鼻、口、顎、etc...)
*   68個のランドマークがある
*   ランドマークの距離を測り比較し顔判定する




# **顔検出の種類**

## Haar-like特徴量



*   白と黒のパターンを組み合わせて特徴量を取得する
*   長所：CPUでほぼリアルタイムに操作、画像サイズに関係なく顔を検出できる
*   短所：顔でない場所も検出してしまう、露出の多い画像や正面画像以外は機能しない




## Deep Neural Network



*   長所：他のモデルよりも正確、CPUでリアルタイムに実行、様々な顔向きに対応
*   短所：処理が遅い、SSDで構成されているがソースは未公開



## HoG特徴量

顔の部品(HoG特徴)を抽出

*   分割されたセルから勾配の大きさと方向を計算
*   ヒストグラムで特徴抽出
*   抽出されたHoG特徴から、SVMで分類
*   輝度の勾配方向をヒストグラム化した特徴量、物体の輪郭部分が強調されて確認できる



HoG特徴量

*   Histograms of Oriented Gradientsは、画像の物体検出で使われる特徴記述子
*   セルグリッド上から画像の局所的な輝度と輝度の勾配方向を計算
*   その勾配方向を輝度分布のヒストグラムにしたものを特徴量とする
*   画像スケールに対してロバスト




*   画像を31x31に区切る
*   セル領域ごとに勾配を計算
*   入力画像を水平方向、垂直方向に一次微微分フィルターで微分
*   アークタンジェントでラジアンを算出して勾配方向を求める




度数変換：一次微分で微分された行と列の値をアークタンジェントでラジアンを計算

ラジアンを角度に変換、角度を180度で割った時の余りを新しい角度とする

水平方向、垂直方向に微分した値のノルムをとって、勾配の規模(強度)とする

0~180度まで20度、9方向に分割



*   長所：CPUで動作しDNNより高速、斜めでも機能する、軽量モデル、をしゅつの強い画像でも動作

*   短所：最小の顔サイズが80x80で訓練される、下向きや上向きなどには機能しない



# **ブラウザで確認してみよう**

## ライブラリのインポート

In [2]:
!pip install face_recognition==1.2.3
!pip install dlib==19.18.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 顔を検出してみよう

In [3]:
import face_recognition
from PIL import Image, ImageDraw

In [4]:
image = face_recognition.load_image_file("/content/sakatai.jpeg")
face_locations = face_recognition.face_locations(image, model="hog")
face_locations # 右上(y1, x1), 左下(y2, x2)

[(741, 1380, 1891, 230)]

In [5]:
image

array([[[188, 188, 152],
        [187, 187, 151],
        [188, 188, 152],
        ...,
        [182, 173, 156],
        [180, 171, 154],
        [181, 172, 155]],

       [[181, 181, 145],
        [179, 179, 143],
        [178, 178, 142],
        ...,
        [183, 174, 157],
        [182, 173, 156],
        [183, 174, 157]],

       [[174, 174, 138],
        [167, 167, 131],
        [161, 161, 125],
        ...,
        [185, 176, 159],
        [186, 177, 160],
        [186, 177, 160]],

       ...,

       [[ 45,  41,  38],
        [ 47,  43,  40],
        [ 53,  49,  46],
        ...,
        [128, 112,  89],
        [129, 113,  90],
        [129, 113,  90]],

       [[ 42,  38,  35],
        [ 43,  39,  36],
        [ 49,  45,  42],
        ...,
        [127, 111,  88],
        [135, 119,  96],
        [136, 120,  97]],

       [[ 41,  37,  34],
        [ 39,  35,  32],
        [ 45,  41,  38],
        ...,
        [130, 114,  91],
        [129, 113,  90],
        [131, 115,  92]]

In [6]:
# 顔部分に枠をつける
def face_detection(image, face_locations):
    face_locations = (face_locations[0][1], face_locations[0][0], face_locations[0][3], face_locations[0][2])
    im = Image.fromarray(image)
    draw = ImageDraw.Draw(im)
    draw.rectangle(face_locations, fill=None, outline=(255, 0, 0), width=5)
    return im

In [ ]:
face_detection(image, face_locations)

## 類似度検証

In [8]:
image1 = face_recognition.load_image_file('/content/sakatai.jpeg')
image2 = face_recognition.load_image_file('/content/sakatai2.jpeg')

In [9]:
encoding1 = face_recognition.face_encodings(image1)[0]
encoding2 = face_recognition.face_encodings(image2)[0]

In [10]:
face_recognition.compare_faces([encoding1], encoding2, tolerance=0.5)

[True]

## ランドマークの検出

In [12]:
def landmark_point(xy1, size=5):
    # xy1[0]+size=半径
    xy2 = xy1[0]+size, xy1[1]+size
    return [(xy1), (xy2)]

In [14]:
image = face_recognition.load_image_file('/content/sakatai.jpeg')
face_landmark_list = face_recognition.face_landmarks(image)

In [15]:
# 画像を読み込んだファイルの配列を画像形式に変換
im = Image.fromarray(image)

# 画像を描画できるようにする
draw = ImageDraw.Draw(im)

for face_landmark in face_landmark_list[0]:
    for xy in face_landmark_list[0][face_landmark]:
        # 楕円を描くコマンド
        draw.ellipse(landmark_point(xy, size=5), outline=(255, 0, 0), fill=(255, 0, 0))

In [ ]:
im

## ランドマークのキーと値

In [19]:
# 画像を読み込んだファイルの配列を画像形式に変換
im = Image.fromarray(image)

# 画像を描画できるようにする
draw = ImageDraw.Draw(im)

for face_landmark in face_landmark_list[0]:
    print('face_landmark', face_landmark)
    for xy in face_landmark_list[0][face_landmark]:
        print('xy', xy)
        # 楕円を描くコマンド
        draw.ellipse(landmark_point(xy, size=5), outline=(255, 0, 0), fill=(255, 0, 0))

face_landmark chin
xy (243, 980)
xy (250, 1126)
xy (265, 1266)
xy (293, 1397)
xy (339, 1529)
xy (412, 1647)
xy (506, 1749)
xy (617, 1833)
xy (750, 1863)
xy (880, 1843)
xy (987, 1766)
xy (1075, 1666)
xy (1154, 1542)
xy (1206, 1408)
xy (1232, 1261)
xy (1242, 1112)
xy (1245, 969)
face_landmark left_eyebrow
xy (338, 871)
xy (399, 794)
xy (501, 776)
xy (603, 794)
xy (697, 839)
face_landmark right_eyebrow
xy (852, 863)
xy (933, 814)
xy (1028, 788)
xy (1122, 803)
xy (1170, 878)
face_landmark nose_bridge
xy (772, 972)
xy (776, 1080)
xy (779, 1183)
xy (781, 1289)
face_landmark nose_tip
xy (662, 1347)
xy (716, 1366)
xy (774, 1382)
xy (829, 1369)
xy (882, 1352)
face_landmark left_eye
xy (449, 971)
xy (504, 942)
xy (574, 945)
xy (633, 990)
xy (569, 1006)
xy (499, 1006)
face_landmark right_eye
xy (882, 999)
xy (948, 955)
xy (1019, 949)
xy (1070, 975)
xy (1023, 1015)
xy (952, 1017)
face_landmark top_lip
xy (589, 1543)
xy (660, 1505)
xy (722, 1486)
xy (775, 1505)
xy (823, 1489)
xy (874, 1512)
xy (930